In [ ]:
!pip install hazm

In [ ]:
import numpy as np
import pandas as pd
import re
import math
from hazm import *
from collections import Counter

normalizer = Normalizer()
stemmer = Stemmer()
lemmatizer = Lemmatizer()

dfTrain = pd.read_csv('books_train.csv')
dfTest = pd.read_csv('books_test.csv')

def cleaning(SentenceList):

  removes = ['.', ':', '،', '(',')','«', '»','"','-','!','[',']', '؛','ـ','*','است','بود','شد','از',',']
  signTrans = str.maketrans("", "", ''.join(removes))

  SentenceList = [item.translate(signTrans) for item in SentenceList]

  numberTrans = str.maketrans("۰۱۲۳۴۵۶۷۸۹", "0123456789")
  SentenceList = [item.translate(numberTrans) for item in SentenceList]
  SentenceList = [re.sub(r'\d', '', item) for item in SentenceList]

  SentenceList = [normalizer.normalize(item) for item in SentenceList]

  return SentenceList

def Lprint(printList):
  i= 0
  for item in printList:
    print(i)
    print(item)
    print('')
    i= i+1

def BowMaker(category, wholeWords):
    wordsList = word_tokenize(''.join(cleaning(dfTrain[dfTrain['categories'] == category]['description'].tolist())))
    wordsList = [lemmatizer.lemmatize(item) for item in wordsList]
    wordCounts = Counter(wordsList)
    counts = [wordCounts[word] for word in wholeWords]
    return counts


In [ ]:
desTrainList = cleaning(dfTrain['description'].tolist())
desTestList = cleaning(dfTest['description'].tolist())

wholeText = ''.join(desTrainList)
wholeWords = word_tokenize(wholeText)
wholeWords = [lemmatizer.lemmatize(item) for item in wholeWords]
wholeWords = list(set(wholeWords))

In [ ]:
islam = BowMaker('کلیات اسلام', wholeWords)
story = BowMaker('داستان کوتاه', wholeWords)
novel = BowMaker('رمان', wholeWords)
kids = BowMaker('داستان کودک و نوجوانان', wholeWords)
job = BowMaker('مدیریت و کسب و کار', wholeWords)
sociaty = BowMaker('جامعه‌شناسی', wholeWords)

In [ ]:
answer = []
for typ in dfTest['categories'].tolist() :
    if typ == 'کلیات اسلام':
        answer.append(1)
    elif typ == 'داستان کوتاه':
        answer.append(2)
    elif typ == 'رمان':
        answer.append(3)
    elif typ == 'داستان کودک و نوجوانان':
        answer.append(4)
    elif typ == 'مدیریت و کسب و کار':
        answer.append(5)
    elif typ == 'جامعه‌شناسی':
        answer.append(6)

In [ ]:
category1 = []
sums = [sum(islam),sum(story),sum(novel),sum(kids),sum(job),sum(sociaty)]

for sentence in desTestList:
  chance = [[] for _ in range(6)]
  words = word_tokenize(sentence)
  words = [lemmatizer.lemmatize(item) for item in words]

  for word in words:
    if word in wholeWords:
      indexInWhole = wholeWords.index(word)
      if islam[indexInWhole] != 0:
        chance[0].append(math.log((islam[indexInWhole])/(sums[0])))
      if story[indexInWhole] != 0:
        chance[1].append(math.log((story[indexInWhole])/(sums[1])))
      if novel[indexInWhole] != 0:
        chance[2].append(math.log((novel[indexInWhole])/(sums[2])))
      if kids[indexInWhole] != 0:
        chance[3].append(math.log((kids[indexInWhole])/(sums[3])))
      if job[indexInWhole] != 0:
        chance[4].append(math.log((job[indexInWhole])/(sums[4])))
      if sociaty[indexInWhole] != 0:
        chance[5].append(math.log((sociaty[indexInWhole])/(sums[5])))

  chances = [sum(chance_i) for chance_i in chance]
  category1.append(chances.index(max(chances))+1)

result = [a == b for a, b in zip(answer, category1)]

print(category1)
print(answer)
print((result.count(True)/450)*100)

[4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 1, 2, 4, 5, 4, 4, 5, 4, 6, 4, 4, 4, 4, 1, 4, 4, 4, 5, 4, 5, 4, 5, 4, 6, 5, 6, 4, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 4, 1, 4, 4, 5, 4, 4, 1, 5, 1, 4, 4, 4, 1, 4, 4, 1, 5, 6, 4, 1, 4, 4, 5, 1, 1, 4, 4, 4, 1, 4, 2, 4, 4, 4, 4, 1, 2, 4, 4, 5, 2, 4, 5, 4, 4, 5, 4, 3, 4, 4, 1, 4, 4, 4, 5, 4, 4, 1, 4, 4, 4, 4, 4, 4, 6, 5, 4, 4, 4, 4, 4, 3, 4, 5, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 6, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 1, 3, 4, 4, 1, 4, 1, 4, 3, 4, 4, 4, 4, 4, 1, 4, 4, 3, 4, 4, 4, 4, 5, 4, 1, 4, 4, 4, 2, 4, 1, 5, 5, 5, 4, 4, 5, 4, 4, 1, 4, 4, 4, 4, 1, 3, 4, 5, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 5, 4, 5, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 5, 4, 5, 4, 5, 4, 4, 4, 1, 4, 4, 4, 5, 4, 5, 5, 4, 4, 1, 4, 1, 4, 4, 4, 1, 4, 4, 4, 4, 1, 1, 4, 4, 2, 4, 4, 4, 4, 1, 4, 5, 4, 4, 4, 5, 4, 5, 4, 4, 2, 4, 1, 4, 

In [ ]:
category = []
a= 0.1
sums = [sum(islam),sum(story),sum(novel),sum(kids),sum(job),sum(sociaty)]

for sentence in desTestList:
  chance = [[] for _ in range(6)]
  words = word_tokenize(sentence)
  words = [lemmatizer.lemmatize(item) for item in words]
  wordsNum = len(words)


  for word in words:
    if word in wholeWords:
      indexInWhole = wholeWords.index(word)
      chance[0].append(math.log((islam[indexInWhole]+a)/(sums[0]+a*wordsNum)))
      chance[1].append(math.log((story[indexInWhole]+a)/(sums[1]+a*wordsNum)))
      chance[2].append(math.log((novel[indexInWhole]+a)/(sums[2]+a*wordsNum)))
      chance[3].append(math.log((kids[indexInWhole]+a)/(sums[3]+a*wordsNum)))
      chance[4].append(math.log((job[indexInWhole]+a)/(sums[4]+a*wordsNum)))
      chance[5].append(math.log((sociaty[indexInWhole]+a)/(sums[5]+a*wordsNum)))

  chances = [sum(chance_i) for chance_i in chance]
  category.append(chances.index(max(chances))+1)

result = [a == b for a, b in zip(answer, category)]

print(category)
print(answer)
print((result.count(True)/450)*100)

[2, 2, 1, 3, 4, 6, 1, 2, 6, 1, 5, 5, 3, 2, 2, 3, 3, 5, 5, 5, 4, 5, 5, 6, 4, 2, 4, 3, 4, 2, 5, 2, 1, 2, 6, 1, 1, 2, 6, 5, 3, 1, 3, 6, 5, 2, 2, 5, 3, 5, 6, 6, 5, 3, 6, 2, 6, 6, 6, 5, 2, 4, 1, 2, 6, 5, 1, 2, 6, 4, 3, 4, 6, 1, 6, 2, 2, 6, 2, 6, 4, 2, 1, 3, 4, 2, 4, 3, 2, 6, 2, 3, 5, 3, 5, 3, 4, 3, 5, 6, 2, 1, 4, 6, 5, 1, 1, 4, 5, 5, 5, 6, 4, 3, 3, 5, 4, 3, 6, 1, 4, 4, 3, 3, 3, 2, 4, 2, 3, 1, 1, 6, 2, 5, 4, 3, 6, 1, 5, 2, 4, 3, 5, 4, 6, 6, 1, 6, 3, 2, 3, 4, 6, 6, 2, 1, 5, 3, 2, 1, 2, 6, 2, 1, 2, 2, 6, 1, 3, 3, 1, 2, 2, 1, 3, 1, 2, 2, 2, 4, 3, 3, 3, 3, 5, 5, 1, 5, 5, 6, 3, 6, 1, 3, 3, 5, 6, 3, 6, 1, 5, 6, 1, 5, 6, 4, 1, 3, 3, 2, 6, 2, 5, 6, 1, 2, 6, 2, 3, 2, 2, 2, 5, 6, 1, 6, 3, 1, 1, 2, 3, 6, 2, 4, 2, 4, 1, 2, 4, 1, 2, 3, 1, 3, 6, 2, 1, 1, 1, 3, 4, 5, 3, 4, 6, 5, 3, 1, 5, 5, 1, 4, 3, 4, 5, 3, 5, 5, 1, 6, 4, 6, 2, 6, 5, 3, 2, 3, 6, 6, 5, 6, 4, 1, 4, 1, 6, 3, 4, 5, 2, 6, 4, 2, 3, 4, 6, 3, 6, 2, 4, 5, 6, 1, 4, 1, 5, 6, 1, 2, 3, 1, 6, 5, 2, 6, 6, 6, 2, 6, 4, 5, 2, 5, 4, 1, 4, 6, 1, 4, 5, 4, 3, 